# Optimization Homework 4
1. A cylindrical tank has its height $=h$ and the radius of the top and bottom $=r$ (both in meters). What $h$ and $r$ values will minimize the total surface area (including the top, bottom, and the side) of this cylindrical tank. We are also told that the volume of this tank has to be fixed (i.e. $\pi r^2h= 20 m^3)$. Please reformulate the objective function to include a penalty term, and solve the problem numerically using any method we have learned (e.g. the Nelder-Mead method or DFP method).

The surface area for a cylinder is $f(h, r) = 2\pi r(h+r)$, and we are given the constraint $g(h, r) = \pi r^2h = 20$. 
We can reformulate the objective function to an *interior* penalty method as
$$\varphi(h,r, r_k) = f(h, r) + r_kg(h,r)^2 = 2\pi r(h+r) + r_k (\pi r^2h-20)^2$$

In [ ]:
import math
import numpy as np

# Objective function with penalty term
def obj(h, r, rk=1):
    first = 2 * math.pi * r * (h + r)
    sec = rk * math.pow((math.pi * (r ** 2) * h - 20), 2)
    return first + sec
    

## Nelder Mead
**Note**: In the book, it's suggested that for equality constraints, we have $H(r_k) = \frac{1}{\sqrt{r_k}}$, whereas in the class notes we just use $r_k$. I think both should be ok as long as we choose $r_k$ well each iteration.

In [ ]:
tol = .01
# Initial simplex
simp = np.array([[0, 0],
                 [1, 0]
                 [1, 1]
                ])
while 